<a href="https://colab.research.google.com/github/jhodgens17/audio_classification/blob/main/MVP_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import packages
import os.path
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers.experimental import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
target_words = ['down', 'go', 'left', 'no', 'off',
           'on', 'right', 'stop', 'up', 'yes']

In [3]:
unknown = ['dog', 'marvin']

In [4]:
all_words = target_words + unknown

In [5]:
len(all_words)

12

In [6]:
all_files = {}
for word in all_words:
  path = '/content/drive/MyDrive/train/audio/'+word
  file_list = os.listdir(path)
  all_files[word] = file_list

In [7]:
len(all_files)

12

In [8]:
for word in all_files:
  print(len(all_files[word]))

2359
2372
2353
2375
2357
2367
2367
2380
2375
2377
1746
1746


In [9]:
total = 0
for word in all_files:
  total = total + len(all_files[word])

In [10]:
total

27174

In [11]:
all_paths = {}
for word in all_files:
  paths = []
  for file in all_files[word]:
    path = '/content/drive/MyDrive/train/audio/' + word + '/' + file
    paths.append(path)
  all_paths[word] = paths

In [12]:
len(all_paths)

12

In [13]:
total_paths = 0
for word in all_paths:
  total_paths = total_paths + len(all_paths[word])

In [14]:
total_paths

27174

In [15]:
y = []
all_paths = []
for target in all_files:
  if target in target_words:
    for file in all_files[target]:
      path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
      all_paths.append(path)
      y.append(target)
  else:
    for file in all_files[target]:
      path = '/content/drive/MyDrive/train/audio/' + target + '/' + file
      all_paths.append(path)
      y.append('unknown')

In [16]:
len(y)

27174

In [17]:
len(all_paths)

27174

In [18]:
y[20000]

'up'

In [19]:
all_paths[20000]

'/content/drive/MyDrive/train/audio/up/c661be6e_nohash_0.wav'

In [20]:
#Getting all of the files
files_ds = tf.data.Dataset.from_tensor_slices(all_paths)

In [21]:
#Defining function to get waveform for each clip
def get_waveform(path):
  audio_binary = tf.io.read_file(path)
  audio, sample_rate = tf.audio.decode_wav(audio_binary)
  waveform = tf.squeeze(audio, axis=-1)
  return waveform

In [22]:
#Getting the waveform for each file
waveform_ds = files_ds.map(get_waveform)

In [23]:
#Defining a function that makes all audio clips same length and gets spectrogram
def get_spectrogram(waveform):
  zero_padding = tf.zeros([16000] - tf.shape(waveform), dtype=tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(equal_length, frame_length=255, frame_step=128)
  spectrogram = tf.abs(spectrogram)
  return spectrogram

In [24]:
#Getting the spectrogram for each file
spectrogram_ds = waveform_ds.map(get_spectrogram)

In [25]:
X = []
for spec in spectrogram_ds:
  X.append(spec.numpy())

In [26]:
X = np.array(X)
y = np.array(y)

In [27]:
X.shape

(27174, 124, 129)

In [28]:
y_d = pd.get_dummies(y)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y_d)

In [30]:
model = Sequential()

In [31]:
for spectrogram in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)

Input shape: (124, 129)


In [32]:
num_labels = 11

In [33]:
model.add(Input(shape=input_shape))
model.add(Flatten())
model.add(Dense(12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(num_labels))

In [34]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [35]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
319/319 [==============================] - 3s 4ms/step - loss: 0.6738 - accuracy: 0.1424
Epoch 2/50
319/319 [==============================] - 1s 4ms/step - loss: 0.5895 - accuracy: 0.1954
Epoch 3/50
319/319 [==============================] - 1s 5ms/step - loss: 0.5265 - accuracy: 0.2102
Epoch 4/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4983 - accuracy: 0.2184
Epoch 5/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4954 - accuracy: 0.2237
Epoch 6/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4988 - accuracy: 0.2328
Epoch 7/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4946 - accuracy: 0.2386
Epoch 8/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4977 - accuracy: 0.2400
Epoch 9/50
319/319 [==============================] - 1s 4ms/step - loss: 0.4919 - accuracy: 0.2417
Epoch 10/50
319/319 [==============================] - 1s 4ms/step - loss: 0.5020 - accuracy: 0.2484

In [36]:
y_pred = np.argmax(model.predict(X_test), axis = 1)

In [37]:
y_test_arr = y_test.idxmax(axis = 1)

In [38]:
y_test.columns

Index(['down', 'go', 'left', 'no', 'off', 'on', 'right', 'stop', 'unknown',
       'up', 'yes'],
      dtype='object')

In [39]:
tar_dict = {0:'down', 1:'go', 2:'left', 3:'no', 4:'off',
            5:'on', 6:'right', 7:'stop', 8:'unknown', 9:'up', 10: 'yes'}

In [40]:
y_hat = []
for value in y_pred:
  y_hat.append(tar_dict[value])

In [41]:
y_hat_arr = np.array(y_hat)

In [42]:
accuracy_score(y_test_arr, y_hat_arr)

0.2749484839564321

In [43]:
model.save('/content/drive/MyDrive/model2')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/model2/assets


In [51]:
model.predict(X_test)[1]

array([ -3.339738  , -15.19149   ,  -9.961892  , -13.914973  ,
         0.55936277,  -4.6677656 , -13.601881  , -10.032476  ,
       -12.749056  ,  -0.19837065,  -3.2556572 ], dtype=float32)

In [ ]:
array([-1.2117121 , -6.394063  , -0.43727398, -5.1449018 , -2.7971497 ,
       -0.11864133, -1.4343631 , -5.095385  , -6.809304  , -3.8131297 ,
        1.3779329 ], dtype=float32)

In [54]:
y_hat[1]

'off'